In [1]:
from pathlib import Path
import os

In [2]:
pip install kaggle

In [9]:
from fastai.imports import *

In [5]:
from google.colab import files
files.upload()  # Upload the kaggle.json file here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"surajjha96","key":"fbbbdadb1d757ce8e7a20bfdf79bdc5c"}'}

In [7]:
!mkdir -p /root/.config/kaggle
!mv kaggle.json /root/.config/kaggle/
!chmod 600 /root/.config/kaggle/kaggle.json

In [8]:
import zipfile, kaggle
path = Path('us-patent-phrase-to-phrase-matching')
kaggle.api.competition_download_cli(str(path))
zipfile.ZipFile(f'{path}.zip').extractall(path)

In [10]:
path.ls()

(#3) [Path('us-patent-phrase-to-phrase-matching/test.csv'),Path('us-patent-phrase-to-phrase-matching/sample_submission.csv'),Path('us-patent-phrase-to-phrase-matching/train.csv')]

In [18]:
train_df = pd.read_csv(path/'train.csv')
sample_submission = pd.read_csv(path/'sample_submission.csv')
eval_df = pd.read_csv(path/'test.csv')

In [19]:
sample_submission.head()

,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


In [20]:
train_df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [22]:
train_df['anchor'].describe()

,anchor
count,36473
unique,733
top,component composite coating
freq,152


In [23]:
train_df['context'].describe()

,context
count,36473
unique,106
top,H01
freq,2186


In [24]:
train_df['target'].describe()

,target
count,36473
unique,29340
top,composition
freq,24


In [26]:
train_df.score.describe()

,score
count,36473.000000
mean,0.362062
std,0.258335
min,0.000000
25%,0.250000
50%,0.250000
75%,0.500000
max,1.000000


In [31]:
from torch.utils.data import DataLoader
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer
!pip install -q datasets
import datasets
from datasets import load_dataset, Dataset, DatasetDict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

In [32]:
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [34]:
model_nm = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_nm)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

In [39]:
sep = tokenizer.sep_token
train_df['inputs'] = train_df.context + sep + train_df.anchor + sep + train_df.target

In [40]:
ds = Dataset.from_pandas(train_df).rename_column('score', 'label')
eval_ds = Dataset.from_pandas(eval_df)

In [42]:
def tok_func(x): return tokenizer(x["inputs"])
tok_func(ds[0])

{'input_ids': [1, 336, 5753, 2, 47284, 2, 47284, 265, 6435, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [48]:
inps = "anchor","target","context"
tok_ds = ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id'))

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [49]:
anchors = train_df.anchor.unique()
np.random.seed(42)
np.random.shuffle(anchors)
print(anchors[:5])
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]
is_val = np.isin(train_df.anchor, val_anchors)
idxs = np.arange(len(train_df))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

['time digital signal' 'antiatherosclerotic' 'filled interior'
 'dispersed powder' 'locking formation']


(9116, 27357)

In [50]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [51]:
def corr(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [54]:
lr,bs = 8e-5,128
wd,epochs = 0.01,4
def get_trainer(dds):
    args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=epochs, weight_decay=wd, report_to='none')
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
    return Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                   tokenizer=tokenizer, compute_metrics=corr)
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr)

In [55]:
trainer.train()

Step,Training Loss
500,0.032200


TrainOutput(global_step=856, training_loss=0.02431747846514265, metrics={'train_runtime': 224.2127, 'train_samples_per_second': 488.054, 'train_steps_per_second': 3.818, 'total_flos': 468342161238420.0, 'train_loss': 0.02431747846514265, 'epoch': 4.0})